In [3]:
import snowflake.connector
import pandas as pd
from dotenv import load_dotenv
import os

# Charger les variables d'environnement
load_dotenv()

# Configuration Snowflake
config = {
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

In [4]:
conn = snowflake.connector.connect(**config)
cursor = conn.cursor()
cursor.execute("SELECT CURRENT_DATABASE(), CURRENT_SCHEMA()")
current_db, current_schema = cursor.fetchone()
print(f"\n📂 Base de données actuelle: {current_db}")
print(f"📁 Schéma actuel: {current_schema}")


📂 Base de données actuelle: RAW_DATA_INTERN
📁 Schéma actuel: SILVER


In [7]:
def get_cursor():
    conn = snowflake.connector.connect(**config)
    cursor = conn.cursor()
    return cursor


def get_table(table_name="DEMO_SEG_CLIENT", limit=None):
    if limit != None:
        cursor.execute(f"SELECT * FROM {table_name} LIMIT {limit}")
    else:
        cursor.execute(f"SELECT * FROM {table_name}")
    data = cursor.fetch_pandas_all()
    return data   

get_table()

,ID_H3,NUMERO,REP,NOM_VOIE,CODE_POSTAL,NOM_COMMUNE,FEMME,AGE,PANIER_MOY,RETAIL,WEB,RECENCE
0,8a3968834977fff,23,None,Impasse de la Cadiere,13300,Salon-de-Provence,True,31,132,20,80,84
1,8a5e436932d7fff,21,None,La Souche Calvaire,97180,Sainte-Anne,True,40,75,87,13,200
2,8a5f12a19b0ffff,1880,None,Route de Montabo (RD1),97300,Cayenne,False,42,384,43,57,90
3,8a1fb476ec47fff,4,None,Rue du Bois Jacques,95600,Eaubonne,False,33,157,40,60,78
4,8a1fb4749c57fff,22,None,Rue des Marronniers,93200,Saint-Denis,True,49,31,100,0,269
...,...,...,...,...,...,...,...,...,...,...,...,...
99994,8a1f946a9c57fff,2,None,Chemin des Bartins,03300,Cusset,False,42,99,47,53,48
99995,8a1f9024bd27fff,82,None,Rue Molière,69330,Meyzieu,False,45,75,80,20,230
99996,8a1f9024bd27fff,30,None,Rue des Gadelles,69330,Meyzieu,False,50,116,49,51,129
99997,8a1fb4700717fff,4,None,Rue Pierre et Charles Perret,78300,Poissy,True,49,77,80,20,308


In [18]:
import numpy as np

data = get_table()

data = get_table()
data.drop(['ID_H3','RF'], axis=1, inplace=True)
data.replace({-1: np.nan}, inplace=True)

data.isna().sum()

data.fillna(data.mean(), inplace=True)

In [24]:
## k-means clustering

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

data = get_table()
data.drop(['ID_H3','RF', 'ZONES_HAB_CAT', 'RESTAURANTS_CAT', 'COMMERCES_CAT','EDUCATION_CAT' ], axis=1, inplace=True)
data.replace({-1: np.nan}, inplace=True)
data.fillna(data.mean(), inplace=True)


scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

kmeans = KMeans(n_clusters=4, random_state=42)  

kmeans.fit(scaled_data)

data['cluster'] = kmeans.labels_

statistics_clusters_preview = data.groupby('cluster').mean().to_json(orient='index', indent=2)
statistics_clusters_preview

'{\n  "0":{\n    "AGE":50.0527331966,\n    "PANIER_MOY":50.9135392784,\n    "RETAIL":79.957916013,\n    "WEB":20.042083987,\n    "RECENCE":226.0529443707,\n    "CSP":1.56452878,\n    "FEMME":0.5028055991,\n    "PCT_C21_MEN_FAM_CAT":2.9946301436,\n    "PCT_MEN_PROP_CAT":3.8889216921,\n    "PCT_LOG_AV45_CAT":1.8005439927,\n    "PCT_LOG_45_70_CAT":1.6617149816,\n    "PCT_LOG_70_90_CAT":2.0496266195,\n    "PCT_LOG_AP90_CAT":2.7574869021,\n    "PCT_LOG_SOC_CAT":1.194858357,\n    "REV_MED_CAT":2.6117624582,\n    "INEG_REV_CAT":3.0870871408,\n    "ETABLISSEMENTS_CAT":2.0815608693\n  },\n  "1":{\n    "AGE":45.8011995953,\n    "PANIER_MOY":108.0484174014,\n    "RETAIL":60.3264922677,\n    "WEB":39.6735077323,\n    "RECENCE":142.7976586212,\n    "CSP":1.6690995809,\n    "FEMME":0.5059257118,\n    "PCT_C21_MEN_FAM_CAT":2.6749530279,\n    "PCT_MEN_PROP_CAT":1.7914611162,\n    "PCT_LOG_AV45_CAT":1.4062282209,\n    "PCT_LOG_45_70_CAT":1.8197256864,\n    "PCT_LOG_70_90_CAT":1.7427205029,\n    "PCT_LO

In [25]:
scaled_data

array([[-1.00155122, -0.05962485,  1.16093459, -0.65975838,  0.65975838,
        -0.88903598],
       [-1.00155122,  0.06099389,  0.49745948,  0.04312832, -0.04312832,
        -0.16836117],
       [-1.00155122,  1.50841875, -0.77420115,  1.65563311, -1.65563311,
         1.96321223],
       ...,
       [-1.00155122,  0.54346884,  0.02196899, -0.28764189,  0.28764189,
        -0.32061641],
       [-1.00155122,  0.18161263,  2.37730563, -1.81745413,  1.81745413,
        -1.32550101],
       [-1.00155122,  0.4228501 ,  0.81813912, -0.37033445,  0.37033445,
        -1.26459891]], shape=(99999, 6))

In [24]:
kmeans.labels_

array([1, 1, 2, ..., 1, 3, 1], shape=(99999,), dtype=int32)